# Spotify Playlist Analysis


<span class = "myhighlight">Objective.</span> Using Python, the project goal is to implement a k-means clustering algorithm, a technique often used in machine learning, and use it for data analysis. We write various functions making use of lists, sets, dictionaries, sorting, and graph data structures for computational problem solving and analysis.


In [1]:
import csv
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from operator import index

First, we create a Client Credentials Flow Manager used in server-to-server authentication by passing the necessary parameters to the [Spotify OAuth](https://github.com/spotipy-dev/spotipy/blob/master/spotipy/oauth2.py#L261) class. We provide a client id and client secret to the constructor of this authorization flow, which does not require user interaction.
    

In [2]:
# Set client id and client secret
client_id = '4cf3afdca2d74dc48af9999b1b7c9c61'
client_secret = 'f6ca08ad37bb41a0afab5ca1dc74b208'

# Spotify authentication token
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

Now, we want to get the full details of the tracks of a playlist based on a playlist ID, URI, or URL. The following function takes a playlist and gets information from each individual song.


In [93]:
# Get playlist song features and artist info
def playlist_features(id, artist_id, playlist_id):
    
    # Create Spotify API client variables
    meta = sp.track(id)
    audio_features = sp.audio_features(id)
    artist_info = sp.artist(artist_id)
    playlist_info = sp.playlist(playlist_id)

    # Metadata
    name = meta['name']
    track_id = meta['id']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    artist_id = meta['album']['artists'][0]['id']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # Main artist name, popularity, genre
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]

    # Track features
    acousticness = audio_features[0]['acousticness']
    danceability = audio_features[0]['danceability']
    energy = audio_features[0]['energy']
    instrumentalness = audio_features[0]['instrumentalness']
    liveness = audio_features[0]['liveness']
    loudness = audio_features[0]['loudness']
    speechiness = audio_features[0]['speechiness']
    tempo = audio_features[0]['tempo']
    valence = audio_features[0]['valence']
    key = audio_features[0]['key']
    mode = audio_features[0]['mode']
    time_signature = audio_features[0]['time_signature']
    
    # Basic playlist info
    playlist_name = playlist_info['name']

    return [name, track_id, album, artist, artist_id, release_date, length, popularity, 
            artist_pop, artist_genres, acousticness, danceability, 
            energy, instrumentalness, liveness, loudness, speechiness, 
            tempo, valence, key, mode, time_signature, playlist_name]

Choose a specific playlist to analyze by copying the URL from the Spotify Player interface. Using that link, the following code uses the playlist_tracks method to retrieve a list of IDs and corresponding artists for each track from the playlist. 



In [77]:
# Spotify playlist url
playlist_links = ["https://open.spotify.com/playlist/1nvpVNmzL7Vi1pXcQEiaLx?si=a62187de23924f4c"]

track_ids = []
artist_uris = []
playlist_ids = []

for link in playlist_links:
    playlist_URI = link.split("/")[-1].split("?")[0]
    
    # Extract song ids and artists from playlist
    for x1 in sp.playlist_tracks(playlist_URI)["items"]:   
        song = x1['track']    
        
        if song == None:
            continue          
        
        track_ids.append(song["id"])
        artist_uris.append(song["artists"][0]["uri"])
        playlist_ids.append(playlist_URI)
        

--------------------

The following code loops through each track ID in the playlist and extracts additional song information by calling the function we created above. From there, we can create a pandas data frame by passing in the extracted information and giving the column header names we want. 

In [78]:
# Loop over track ids
tracks = []
for i in range(len(track_ids)):
    time.sleep(.5)
    feat = playlist_features(track_ids[i], artist_uris[i], playlist_ids[i])
    tracks.append(feat)

In [94]:
# Loop over track ids
all_tracks = [playlist_features(track_ids[i], artist_ids[i], playlist_ids[i])
              for i in range(len(track_ids))]

In [79]:
# Create dataframe
df = pd.DataFrame(
    tracks, columns=['name', 'track_id', 'album', 'artist', 'artist_id','release_date',
                     'length', 'popularity', 'artist_pop', 'artist_genres',
                     'acousticness', 'danceability', 'energy',
                     'instrumentalness', 'liveness', 'loudness',
                     'speechiness', 'tempo', 'valence', 'key', 'mode',
                     'time_signature', 'playlist'])
# Save to csv file
df.to_csv("data/my_playlist.csv", sep=',')

--------------------------------------------------------


#### Spotify Playlists Data Extraction

In [71]:
spotify_playlists = pd.read_csv('data/spotify_playlists.csv', encoding_errors='ignore', index_col=0, header=0)
spotify_playlists['playlist'].value_counts()

New Music Friday      100
New Pop Picks         100
just hits             100
Hip Hop Controller     99
RapCaviar              51
Today's Top Hits       50
Hot Hits USA           50
Name: playlist, dtype: int64

------------------------------------------------------

### The Data






[**Metadata.**](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-track)
- `name`: The name of the track.
- `album`: The name of the album on which the track appears.
- `artist`: The name of the artist who performed the track.
- `release_date`: The date the album was first released.
- `length`: The track length in milliseconds.
- `popularity`: The popularity of the track. Values are between 0 and 100. The popularity is calculated by an algorithm based on the total number of plays the track has had and how recent those plays are.


[**Artists.**](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artist)
- `artist_pop`: The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist's popularity is calculated from the popularity of all the artist's tracks.
- `artist_genres`: A list of the genres the artist is associated with.


[**Audio Features.** ](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features)
- `acousticness`: A confidence measure from 0.0 to 1.0 of whether the track is acoustic.
- `danceability`: Describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm, beat strength, and regularity.
- `energy`: A measure from 0.0 to 1.0 that represents a perceptual measure of intensity and activity.
- `instrumentalness`: Predicts whether a track contains no vocals. The closer the value is to 1.0, the more likely the track contains no vocal content.
- `liveness`: Detects the presence of an audience in the recording. Higher values represent an increased probability that the track was performed live.
- `loudness`: The overall loudness of a track in decibels (dB). Values are averaged across entire track, ranging between -60 and 0 db.
- `speechiness`: Detects the presence of spoken words in a track. The more speech-like the recording, the closer to 1.0.
- `tempo`: The overall estimated speed or pace of a track in beats per minute (BPM).
- `valence`: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. High valence sound more positive (e.g. happy, cheerful, euphoric).
- `key`: The key the track is in. If no key was detected, the value is -1.
- `mode`: The modality (major or minor) of a track. Major is represented by 1 and minor is 0.
- `time_signature`: An estimated time signature (how many beats are in each measure), ranging from 3 to 7 indicating time signatures of "3/4", to "7/4".



In [98]:
# df = pd.read_csv('playlists.csv', encoding_errors='ignore', index_col=0, header=0)
df.head(2)

,name,track_id,album,artist,artist_id,release_date,length,popularity,artist_pop,artist_genres,...,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature,playlist
0,2 AM,3g3RCV5ImXwzHpKwM2iunc,Pure Infinity,SwaVay,29gIYsdyccGoUc6qgkZeTK,2019-05-24,198577,55,51,"[atl hip hop, indie hip hop, underground hip hop]",...,0.000098,0.362,-12.353,0.0727,126.799,0.184,7,1,4,but my feet in bottega
1,Golden Child,04QWC97Dvd9g0IEDoyUDBX,Lady Wrangler,Shaboozey,3y2cIKLjiOlp1Np37WiUdH,2018-10-05,177773,46,56,[pop rap],...,0.000002,0.360,-8.848,0.2900,151.029,0.365,0,1,4,but my feet in bottega


How many songs do we have?

In [110]:
# Number of rows and columns
rows, cols = df.shape
print(f'Number of songs: {rows}')
print(f'Number of attributes per song: {cols}')

Number of songs: 100
Number of attributes per song: 23


In [111]:
# Get a song string search
def getMusicName(elem):
    return f"{elem['artist']} - {elem['name']}"

# Select song and get track info
anySong = df.loc[15]
anySongName = getMusicName(anySong)
print('name:', anySongName)

name: Lil Yachty - Yacht Club (feat. Juice WRLD)


-----------------------

## Spotify Songs - Similarity Search




Below, we create a query to retrieve similar elements based on Euclidean distance. In mathematics, the Euclidean distance between two points is the length of the line segment between the two points. In this sense, the closer the distance is to 0, the more similar the songs are.



#### [KNN Algorithm](https://www.kaggle.com/code/leomauro/spotify-songs-similarity-search/notebook)


The k-Nearest Neighbors (KNN) algorithm searches for k similar elements based on a query point at the center within a predefined radius. 



In [112]:
def knnQuery(queryPoint, arrCharactPoints, k):
    queryVals = queryPoint.tolist()
    distVals = []
    
    # Copy of dataframe indices and data
    tmp = arrCharactPoints.copy(deep = True)  
    for index, row in tmp.iterrows():
        feat = row.values.tolist()
        
        # Calculate sum of squared differences
        ssd = sum(abs(feat[i] - queryVals[i]) ** 2 for i in range(len(queryVals)))
        
        # Get euclidean distance
        distVals.append(ssd ** 0.5)
        
    tmp['distance'] = distVals
    tmp = tmp.sort_values('distance')
    
    # K closest and furthest points
    return tmp.head(k).index, tmp.tail(k).index

In [113]:
# Execute KNN removing the query point
def querySimilars(df, columns, idx, func, param):
    arr = df[columns].copy(deep = True)
    queryPoint = arr.loc[idx]
    arr = arr.drop([idx])
    return func(queryPoint, arr, param)

**KNN Query Example.** 

Our function allows us to create personalized query points and modify the columns to explore other options. For example, the following code selects a specific set of song attributes and then searches for the $k$ highest values of these attributes set equal to one.

Let's search for  $k=3$  similar songs to a query point $\textrm{songIndex} = 6$. 

In [115]:
# Select song and column attributes
songIndex = 6 # query point
columns = ['acousticness', 'danceability', 'energy', 'speechiness', 'valence','tempo']

# Set query parameters
func, param = knnQuery, 3

# Implement query
response = querySimilars(df, columns, songIndex, func, param)

print("---- Query Point ----")
print(getMusicName(df.loc[songIndex]))
print('---- k = 3 similar songs ----')
for track_id in response[0]:
    track_name = getMusicName(df.loc[track_id])
    print(track_name)
print('---- k = 3 nonsimilar songs ----')
for track_id in response[1]:
    track_name = getMusicName(df.loc[track_id])
    print(track_name)

---- Query Point ----
YG - Sober (feat. Roddy Ricch & Post Malone)
---- k = 3 similar songs ----
Future - WAIT FOR U (feat. Drake & Tems)
Juice WRLD - Fast
The Game - Eazy
---- k = 3 nonsimilar songs ----
Fresco Trey - Key To My Heart
6LACK - Switch
Post Malone - Candy Paint


The code below implements the same idea as above, but queries each track in a given playlist instead of a single defined query point.

In [129]:
similar_count = {} # Similar songs count
nonsimilar_count = {} # Non-similar songs count

for track_index in df.index:
    response = querySimilars(df, columns, track_index, func, param)
    
    # Get similar songs
    for similar_index in response[0]:
        track = getMusicName(df.loc[similar_index])
        if track in similar_count:
            similar_count[track] += 1
        else:
            similar_count[track] = 1
    
    # Get non-similar songs
    for nonsimilar_index in response[1]:
        track = getMusicName(df.loc[nonsimilar_index])
        if track in nonsimilar_count:
            nonsimilar_count[track] += 1
        else:
            nonsimilar_count[track] = 1

NON SIMILAR SONG COUNT:

In [130]:
nonsimilar = dict(sorted(nonsimilar_count.items(), key=lambda item: item[1], reverse=True))

print('---- NON SIMILAR SONG COUNTS ----')
for track_name, track_count in nonsimilar.items():
    if track_count >= 8:
        print(track_name, ':', track_count)

---- NON SIMILAR SONG COUNTS ----
Post Malone - Internet : 55
AG Club - Memphis : 54
Post Malone - Candy Paint : 50
Boslen - DENY (feat. Tyla Yaweh) : 50
6LACK - Switch : 46
Fresco Trey - Key To My Heart : 45


SIMILAR SONG COUNT:

In [132]:
similar = dict(sorted(similar_count.items(), key=lambda item: item[1], reverse=True))

print('---- SIMILAR SONG COUNTS ----')
for track_name, track_count in similar.items():
    if track_count >= 5:
        print(track_name, ':', track_count)

---- SIMILAR SONG COUNTS ----
Lil Mosey - Noticed : 8
Post Malone - Paranoid : 8
Kevin Abstract - Empty : 7
A$AP Rocky - D.M.B. : 7
Shaboozey - Golden Child : 6
Juice WRLD - Rich And Blind : 6
Roddy Ricch - don’t i (feat. Gunna) : 5
Medasin - Go Crazy : 5
YoungBoy Never Broke Again - Home Ain't Home (feat. Rod Wave) : 5
Rexx Life Raj - Moonwalk : 5
Boslen - DENY (feat. Tyla Yaweh) : 5
The Game - Eazy : 5
Toosii - be cautious : 5


-------------------------------------

---------------------------------------------------------------


### Similar Artists Web Visual


First, we want to find the most frequently occurring artist in a given playlist. We use the value_counts function to get a sequence containing counts of unique values sorted in descending order. 


In [133]:
# pandas count distinct values in column
tallyArtists = df.value_counts(["artist", "artist_id"]).reset_index(name='counts')
topArtist = tallyArtists['artist_id'][1]
tallyArtists.head(4)

,artist,artist_id,counts
0,Juice WRLD,4MCBfE4596Uoi2O4DtmEMz,9
1,Post Malone,246dkjvS1zLTtiykXe5h60,9
2,SAINt JHN,0H39MdGGX6dbnnQPt6NQkZ,4
3,Quavo,0VRj0yCOv2FXJNP47XQnx5,3


#### Links Dataset

I can retrieve artist and artist-related data using the following code, passing the artist ID to the artist and artist-related artist functions under the spotipy package. The returned list of similar artists is sorted by similarity score based on the listener data.

In [ ]:
# create links table
a = sp.artist(topArtist)
ra = sp.artist_related_artists(topArtist)

# dictionary of lists 
links_dict = {"source_name":[],"source_id":[],"target_name":[],"target_id":[]};
for artist in ra['artists']:
    links_dict["source_name"].append(a['name'])
    links_dict["source_id"].append(a['id'])
    links_dict["target_name"].append(artist['name'])
    links_dict["target_id"].append(artist['id'])

Let’s take it a step further and query the API for similar artists for those similar to the most frequent artist in the given playlist. In other words, we generate two generations of the most similar artists.

In [ ]:
for i in range(0, 4):
    a = sp.artist(links_dict['target_id'][i])
    ra = sp.artist_related_artists(links_dict['target_id'][i])
    time.sleep(.5)
    for artist in ra['artists']:
        links_dict["source_name"].append(a['name'])
        links_dict["source_id"].append(a['id'])
        links_dict["target_name"].append(artist['name'])
        links_dict["target_id"].append(artist['id'])

# Convert links dict to dataframe
links = pd.DataFrame(links_dict) 

# Export to excel sheet             
links.to_excel("links.xlsx", index = False)

In [ ]:
links.head(3)

#### Points Dataset

In [ ]:
# create "points" table             
all_artist_ids = list(set(links_dict['source_id'] + links_dict['target_id']))

In [ ]:
# dictionary of lists 
points_dict = {"id":[],"name":[],"followers":[],"popularity":[],"url":[],"image":[]};

for id in all_artist_ids:
    time.sleep(.5)
    a = sp.artist(id)
    points_dict['id'].append(id)
    points_dict['name'].append(a['name'])
    points_dict['followers'].append(a['followers']['total'])
    points_dict['popularity'].append(a['popularity'])
    points_dict['url'].append(a['external_urls']['spotify'])
    points_dict['image'].append(a['images'][0]['url'])

# Convert links dict to dataframe
points = pd.DataFrame(points_dict) 

# Export to excel sheet             
points.to_excel("points.xlsx", index = False)

In [ ]:
points.head(3)

#### Flourish Network Graph

The following visualization is based on the [Spotify Similiar Artists API](https://unboxed-analytics.com/data-technology/visualizing-rap-communities-wtih-python-spotifys-api/) article and created with flourish studio.


In [ ]:
%%html

<iframe src='https://flo.uri.sh/visualisation/12232729/embed' title='Interactive or visual content' class='flourish-embed-iframe' frameborder='0' scrolling='no' style='width:100%;height:600px;' sandbox='allow-same-origin allow-forms allow-scripts allow-downloads allow-popups allow-popups-to-escape-sandbox allow-top-navigation-by-user-activation'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/12232729/?utm_source=embed&utm_campaign=visualisation/12232729' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>

------------------------------------------

----------------------------------------------------

## Organized Songs in a Playlist

In [ ]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import cluster, decomposition

In [ ]:
df['name']

songs = pd.read_csv("data/spotify.dat")
labels = songs.values[:,1]
X = songs.values[:,2:]

In [ ]:


kmeans = cluster.AffinityPropagation(preference=-200)
kmeans.fit(X)

predictions = {}
for p,n in zip(kmeans.predict(X),labels):
	if not predictions.get(p):
		predictions[p] = []

	predictions[p] += [n]

for p in predictions:
	print "Category",p
	print "-----"
	for n in predictions[p]:
		print n
	
	print ""